In [1]:
import sys
sys.path.append("./Delta_by_Ducas")
import numpy as np
from math import factorial as fac
from math import log
import itertools
from Kyber import *

In [2]:
'''
Parameter set
n=2, q=17, eta=1, k=1, du=3, dv=1
Then we iterate over s,e,A = 3^2*3^2*17^2 possibilities
m = 2^2 = 4 posiibilities
r,e1,e2 = 3^2*3^2*3^2 = 27 possiibilities
total = 33156 possibilities
'''


eta_int = [-1,0,1]

all_cbds = []
for i in eta_int:
    for j in eta_int:
        all_cbds.append([i,j])

#for i in all_cbds:
#    print(i)

all_uniform = []
for i in range(0,17):
    for j in range(17):
        all_uniform.append([i,j])

# print(len(all_uniform))

all_msgs = []
for i in [0,1]:
    for j in [0,1]:
        all_msgs.append([i,j])

# print(len(all_msgs))

In [3]:


def binomial(x, y):
    """ Binomial coefficient
    :param x: (integer)
    :param y: (integer)
    :returns: y choose x
    """
    try:
        binom = fac(x) // fac(y) // fac(x - y)
    except ValueError:
        binom = 0
    return binom


def cbd_pdf(k, x):
    """ Probability density function of the centered binomial law of param k at x
    :param k: (integer)
    :param x: (integer)
    :returns: p_k(x)
    """
    return binomial(2*k, x+k) / 2.**(2*k)


def uniform_pdf(n, x):
    return 1/n
    


In [4]:

# consider vecs as well! TODO but we have k=1, so ok

def probability_cbd_poly(eta,s):
    assert(len(s) == 2)
    return cbd_pdf(eta,s[0])*cbd_pdf(eta,s[1])

def probability_uniform_poly(q,A):
    assert(len(A) == 2)
    return 1/(q**2)

def probability_key_gen(eta,q,s,A,e):
    return (probability_cbd_poly(eta,e)) * (probability_cbd_poly(eta,s)) * (probability_uniform_poly(q,A))

def probability_encrypt(eta,r,e1,e2):
    return (probability_cbd_poly(eta,r)) * (probability_cbd_poly(eta,e1)) * (probability_cbd_poly(eta,e2))



In [5]:

def poly_mult(p1,p2,q):
    '''
    Multiplication of two polynomials p1 and p2 in R_q
    p1 :: np.array of coefficients length n
    p2 :: np.array of coefficients length n
    q :: modulus of R_q
    '''
    assert(len(p1)==2) #n=2
    assert(len(p2)==2) # n=2
    mult_0 = (p1[0]*p2[0] - p1[1]*p2[1]) % q
    mult_1 = (p1[0]*p2[1] + p1[1]*p2[0]) % q
    return [mult_0,mult_1]


def vec_add(a,b,q):
    '''
    Addition of two vectors of polynomials a and b in R_q^k
    a :: np.array of vector of coefficients
    b :: np.array of vector of coefficients
    q :: modulus of R_q
    '''
    return [(a[0]+b[0])%q,(a[1]+b[1])%q]

def vec_minus(a,b,q):
    '''
    Difference of two vectors of polynomials a and b in R_q^k
    a :: np.array of vector of coefficients
    b :: np.array of vector of coefficients
    q :: modulus of R_q
    '''
    return [(a[0]-b[0])%q,(a[1]-b[1])%q]

def compress(x,q,d):
    '''
    compression of integer x from modulus q to modulus 2^d
    '''
    return round (x * (2**d) / q) % (2**d)

def decompress(x,q,d):
    '''
    decompression of integer x from modulus q to modulus 2^d
    '''
    return round (x * q / (2**d)) % q

def compress_poly(p,q,d):
    '''
    Compression on polynomials in R_q
    '''
    return [compress(p[0],q,d), compress(p[1],q,d)]

def decompress_poly(p,q,d):
    '''
    Decompression on polynomials in R_q
    '''
    return [decompress(p[0],q,d),decompress(p[1],q,d)]

def centered_mod(p,q):
    assert(len(p)==2)
    p = [p[0] % q, p[1] % q]
    c=[0,0]
    if p[0] <= (q // 2):
        c[0] = p[0]
    else: 
        c[0] = p[0]-q
    if p[1] <= (q // 2):
        c[1] = p[1]
    else: 
        c[1] = p[1]-q
    return c

def centered_mod_norm(p,q):
    return np.abs(centered_mod(p,q))

def poly_norm(p,q):
    return np.max(centered_mod_norm(p,q))




In [6]:
def prop(q,du,dv,A,s,e,r,e1,e2,m):
    t = vec_add(poly_mult(A,s, q), e, q)
    u = vec_add (poly_mult(A,r,q), e1, q)
    to_mod_q = [(q+1) // 2, 0]
    qm = poly_mult(to_mod_q, m, q)
    v = vec_add(vec_add(poly_mult(t, r, q), e2, q), qm, q)
    comp_u = compress_poly(u, q, du) 
    comp_v = compress_poly(v, q, dv)
    dc_v = decompress_poly(comp_v, q, dv)
    dc_u = decompress_poly(comp_u, q, du)
    st_dcu = poly_mult(s, dc_u, q)
    dec = vec_minus(dc_v, st_dcu,q)
    w = vec_minus(dec, qm, q) # == 0 if decrypt(encrypt(m)) = m, != 0 otherwise
    q4 = (q+2) // 4
    prop_bool = (poly_norm(w,q) >= q4)
    return prop_bool

def prop_original(q,du,dv,A,s,e,r,e1,e2,m):
    t = vec_add(poly_mult(A,s, q), e, q)
    u = vec_add (poly_mult(A,r,q), e1, q)
    to_mod_q = [(q+1) // 2, 0]
    qm = poly_mult(to_mod_q, m, q)
    v = vec_add(vec_add(poly_mult(t, r, q), e2, q), qm, q)
    comp_u = compress_poly(u, q, du) 
    comp_v = compress_poly(v, q, dv)
    dc_v = decompress_poly(comp_v, q, dv)
    dc_u = decompress_poly(comp_u, q, du)
    st_dcu = poly_mult(s, dc_u, q)
    dec = vec_minus(dc_v, st_dcu,q)
    compress_dec = compress_poly(dec, q, 1)
    w = vec_minus(compress_dec, m, q) # == 0 if decrypt(encrypt(m)) = m, != 0 otherwise
    res = (poly_norm(w,q) != 0)
    return res

In [7]:
def P_inner(q,du,dv,A,s,e,m):
    sum = 0
    for r in all_cbds:
        for e1 in all_cbds:
            for e2 in all_cbds:
                if prop(q,du,dv,A,s,e,r,e1,e2,m):
                    sum = sum + probability_encrypt(1,r,e1,e2) # eta=1
    return sum

def max_msg(q,du,dv,A,s,e):
    max = 0
    for m in all_msgs:
        p = P_inner(q,du,dv,A,s,e,m)
        if p > max:
            max = p
    return max

def P_outer(q,du,dv):
    sum = 0
    for A in all_uniform:
        for s in all_cbds:
            for e in all_cbds:
                sum = sum + max_msg(q,du,dv,A,s,e)* probability_key_gen(1,q,s,A,e) # eta=1
    return sum



In [8]:
def P_inner_original(q,du,dv,A,s,e,m):
    sum = 0
    for r in all_cbds:
        for e1 in all_cbds:
            for e2 in all_cbds:
                if prop_original(q,du,dv,A,s,e,r,e1,e2,m):
                    sum = sum + probability_encrypt(1,r,e1,e2) # eta=1
    return sum

def max_msg_original(q,du,dv,A,s,e):
    max = 0
    for m in all_msgs:
        p = P_inner_original(q,du,dv,A,s,e,m)
        if p > max:
            max = p
    return max

def P_outer_original(q,du,dv):
    sum = 0
    for A in all_uniform:
        for s in all_cbds:
            for e in all_cbds:
                sum = sum + max_msg_original(q,du,dv,A,s,e)* probability_key_gen(1,q,s,A,e) # eta=1
    return sum


In [9]:
n=2 # hardcoded
q=17
eta=1 # hardcoded
k=1 #  hardcoded
du=3
dv=1

ps_test = KyberParameterSet(n, k, eta, eta, q, q, 2**du,2**dv)

F, delta = p2_cyclotomic_error_probability(ps_test)

delta_prime = P_outer(q,du,dv)
original = P_outer_original(q,du,dv)

print(f'delta = {delta}\n delta\' = {delta_prime}\n correct error = {original}\n delta >= error is {delta >= original}\n delta\' >= error is {delta_prime >= original}')

delta = 0.21099713754683047
 delta' = 0.26724520356596815
 correct error = 0.22346470182742278
 delta >= error is False
 delta' >= error is True
